In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  %pip install tf-nightly
except Exception:
  pass
%pip install tensorflow-datasets

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
from tensorflow import keras
import tensorflow_datasets as tfds
from keras.utils import pad_sequences
import numpy as np
import matplotlib.pyplot as plt
import re,nltk
print(tf.__version__)

     |████████████████████████████████| 448.7MB 35kB/s 
     |████████████████████████████████| 5.9MB 40.4MB/s 
     |████████████████████████████████| 4.0MB 44.1MB/s 
     |████████████████████████████████| 471kB 50.7MB/s 
     |████████████████████████████████| 4.2MB 42.2MB/s 
     |████████████████████████████████| 1.3MB 41.9MB/s 
     |████████████████████████████████| 4.9MB 43.4MB/s 
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.4.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement grpcio~=1.32.0, but you'll have grpcio 1.38.0 which is incompatible.
ERROR: tensorflow 2.4.1 has requirement h5py~=2.10.0, but you'll have h5py 3.1.0 which is incompatible.
  Found existing installation: grpcio 1.32.0
    Uninstalling grpcio-1.32.0:
      Successfully uninstalled grpcio-1.32.0
  Found existing installation: h5py 2.10.0
    Uninstalling h5py-2.10.0:
      Successfully uninstalled h5py-2.10.0
  Found existing installation: gast 0.3.3
    Uninstal

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2021-05-21 18:04:31--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv.2’

train-data.tsv.2    100%[===================>] 349.84K  --.-KB/s    in 0.04s   

2021-05-21 18:04:31 (8.72 MB/s) - ‘train-data.tsv.2’ saved [358233/358233]

--2021-05-21 18:04:31--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.3.33, 172.67.70.149, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.3.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv.2’

valid-data.tsv.2    100%[========

In [ ]:
header=['s_class',"msg"]

In [ ]:
df_train=pd.read_csv(train_file_path,delimiter='\t',quoting=3,names=header)
df_test=pd.read_csv(test_file_path,delimiter='\t',quoting=3,names=header)

In [ ]:
df_train.shape

(4179, 2)

In [ ]:


df_train=df_train[['msg','s_class']]
df_test=df_test[['msg','s_class']]
df_train['s_class'] = df_train['s_class'].apply({'ham':0,'spam':1}.get) 
df_test['s_class'] = df_test['s_class'].apply({'ham':0,'spam':1}.get) 


y_train=np.array(df_train.pop('s_class'))
y_test=np.array(df_test.pop('s_class'))_


In [ ]:
corp_train=[]
corp_test=[]
for i in range(4179):
  rvw=re.sub('[^A-Za-z0-9]',' ',df_train['msg'][i])
  rvw=rvw.lower()
  rvw=rvw.split()
  stem_rvw=nltk.stem.porter.PorterStemmer()
  rvw=[stem_rvw.stem(word) for word in rvw]
  rvw=' '.join(rvw)
  corp_train.append(rvw)
  if(i<1392):
    j=i
    test_rvw=re.sub('[^A-Za-z0-9]',' ',df_test['msg'][j])
    test_rvw=test_rvw.lower()
    test_rvw=test_rvw.split()
    test_stem=nltk.stem.porter.PorterStemmer()
    test_rvw=[test_stem.stem(test_words) for test_words in test_rvw]
    test_rvw=' '.join(test_rvw)
    corp_test.append(test_rvw)

In [ ]:
from keras.preprocessing.text import Tokenizer

token=Tokenizer(num_words=5000)
token.fit_on_texts(corp_train)

x_train=token.texts_to_sequences(corp_train)
x_test=token.texts_to_sequences(corp_test)

voc_size=len(token.word_index)+1

print(corp_train[10])
print(x_train[10])
print(voc_size)


x_train = np.array(pad_sequences(x_train, padding='post', maxlen=90))
x_test = np.array(pad_sequences(x_test, padding='post', maxlen=90))


hello darlin ive finish colleg now so txt me when u finish if u can love kate xxx
[277, 732, 923, 209, 656, 22, 28, 76, 11, 53, 6, 209, 39, 6, 27, 61, 924, 381]
6336


In [ ]:
from keras.models import Sequential
from keras import layers

embedding_dim = 50

model = Sequential()
model.add(layers.Embedding(input_dim=voc_size, 
                           output_dim=embedding_dim, 
                           input_length=90))
model.add(layers.Conv1D(128, 5, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(10, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 90, 50)            316800    
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 86, 128)           32128     
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 10)                1290      
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 11        
Total params: 350,229
Trainable params: 350,229
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(x_train,y_train,
                    epochs=10,
                    verbose=True,
                    validation_data=(x_test,y_test), batch_size=10)

Epoch 1/10
418/418 [==============================] - 5s 11ms/step - loss: 0.1936 - accuracy: 0.9294 - val_loss: 0.0577 - val_accuracy: 0.9806
Epoch 2/10
418/418 [==============================] - 4s 10ms/step - loss: 0.0247 - accuracy: 0.9933 - val_loss: 0.0392 - val_accuracy: 0.9878
Epoch 3/10
418/418 [==============================] - 4s 10ms/step - loss: 0.0068 - accuracy: 0.9983 - val_loss: 0.0406 - val_accuracy: 0.9892
Epoch 4/10
418/418 [==============================] - 4s 10ms/step - loss: 0.0018 - accuracy: 0.9998 - val_loss: 0.0433 - val_accuracy: 0.9878
Epoch 5/10
418/418 [==============================] - 4s 10ms/step - loss: 0.0013 - accuracy: 0.9998 - val_loss: 0.0458 - val_accuracy: 0.9871
Epoch 6/10
418/418 [==============================] - 4s 10ms/step - loss: 4.9917e-04 - accuracy: 1.0000 - val_loss: 0.0498 - val_accuracy: 0.9864
Epoch 7/10
418/418 [==============================] - 4s 10ms/step - loss: 1.3944e-04 - accuracy: 1.0000 - val_loss: 0.0518 - val_accuracy

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  pred=[pred_text]
  df_pred=pd.DataFrame(pred)
  df_pred=df_pred.rename(columns={0:'msg',1:'s_class'})
  corp_pred=[]
  for i in range(0,len(pred)):
    rvw=re.sub('[^A-Za-z0-9]',' ',df_pred['msg'][i])
    rvw=rvw.lower()
    rvw=rvw.split()
    stem_rvw=nltk.stem.porter.PorterStemmer()
    rvw=[stem_rvw.stem(word) for word in rvw]
    rvw=' '.join(rvw)
    corp_pred.append(rvw)
    seq=token.texts_to_sequences(corp_pred)
    seq=pad_sequences(seq,maxlen=90)
    prediction=model.predict(seq)
  if prediction>=0.5:
    prediction=([prediction,'spam'])
  else:
    prediction=([prediction,'ham'])
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[array([[9.4131865e-05]], dtype=float32), 'ham']


In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
